<h2>Izvērtēšana</h2>

In [0]:
from google.colab import drive
import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
drive.mount('/gdrive')
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets/BERT_encoded'

<h3>Nepapildināto jautājumu izvērtēšana</h3>

In [0]:
sakotnejie = pd.read_csv('Encoded_labeled_BERT_big.csv', index_col=0)
print(sakotnejie.shape)
sakotnejie.head(3)

(23217, 769)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,labels
0,0.679284,-0.400574,-0.620375,0.611502,0.734588,0.088327,-0.020892,0.006192,0.369733,-0.123941,0.041169,0.166741,-0.547854,-0.186370,0.325477,0.216363,0.011699,-0.433178,0.244505,0.500606,0.816476,0.751867,-0.625378,-0.128836,0.463435,0.206589,0.397435,0.188761,-1.277055,-0.302809,0.300433,-0.296992,-1.169210,-0.109037,0.322343,-0.114833,-0.247131,0.525235,0.502975,-0.498689,...,-0.197942,-0.370189,0.503417,0.011522,-0.815976,-0.037389,-0.128162,-0.805377,-0.314621,-0.309849,0.627250,0.825871,-1.160942,-0.450810,0.305224,0.011232,-0.966943,-0.035133,-0.325602,1.220090,0.262454,0.373898,-0.218212,0.673751,-0.161502,-0.086320,0.067702,-0.161547,0.207985,0.793658,0.390131,0.355691,0.161900,0.759920,-0.599083,-0.205744,-0.809350,0.006495,-0.047038,2
1,1.461980,-0.126122,-1.126548,-0.067945,0.624471,-0.292858,0.617616,-0.370676,0.418209,-0.718991,0.113030,0.307480,-0.876434,-0.475213,-0.099422,-0.503917,-0.709904,0.231744,-0.626840,0.163213,1.113481,0.780380,-0.024935,0.452357,1.579429,0.149871,0.250889,1.385862,-0.305765,0.298612,-0.635962,-0.453691,-0.620625,1.543632,0.634690,0.352088,-0.383335,1.396535,-0.124010,-0.077733,...,0.696726,-1.240646,-1.314792,-0.778095,-1.237933,0.172612,0.332553,-1.564506,-0.532149,-0.355955,0.688367,0.834911,-0.966829,-0.859371,1.157978,0.918479,-1.147100,-0.161868,-0.588534,1.517730,0.271317,1.058547,0.772102,1.197035,-0.540344,-1.101081,0.919615,-0.055205,0.381695,0.403362,-0.462299,1.075352,-0.547956,1.350157,-1.840111,-1.528479,-1.755251,-0.951623,-0.562593,2
2,0.406492,-0.314168,-0.871162,0.276910,0.393781,0.376317,-0.055553,0.044433,-0.623606,-0.165686,0.080295,0.242634,-0.138413,-0.563997,0.242401,0.562495,0.052240,-0.383717,0.144012,0.737990,0.266702,-0.163825,-0.504020,0.065307,0.195115,0.291265,0.358405,0.116041,-0.536641,-0.480187,0.282035,-0.456998,-0.991170,0.147215,0.287675,0.690125,0.200533,0.746974,0.822502,-0.197148,...,0.111140,-0.295744,0.291161,0.492385,-0.893619,0.023238,-0.162050,-0.693156,-0.549654,-0.148787,0.611298,0.393017,0.051018,-0.552981,0.550665,0.725327,-0.017602,0.131908,-0.320487,1.620777,-0.192227,0.172328,-0.508857,0.468424,0.157647,0.212833,-0.419054,0.728242,-0.027049,1.205139,0.004055,0.415510,0.768602,0.451367,0.053940,-0.648625,-0.613054,-0.268880,-0.104552,2


In [0]:
col_count = len(sakotnejie.columns) - 1
print(col_count)
features = sakotnejie.iloc[:,:col_count]
target = sakotnejie.iloc[:,col_count:]
print(features.shape, target.shape)

768
(23217, 768) (23217, 1)


In [0]:
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets/'
eval_data = pd.read_csv('2020_05_24_BERT_big_izvertesana.csv', index_col=0)
eval_data['labels'].value_counts()
col_count = len(eval_data.columns) - 1
X = eval_data.iloc[:,:col_count]
Y = eval_data.iloc[:,col_count:]
print(X.shape, Y.shape)

/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets
(100, 768) (100, 1)


In [0]:
def transformResults(results):
    mod_results = []
    for result in results:
        out = np.argmax(result)
        mod_results.append(out)
    mod_results = np.array(mod_results)    
    return mod_results 

In [0]:
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
y_test = Y.to_numpy()

model_scores = []
counter = 0
for i in range(10):
  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(features, target,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                     # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                       # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)             # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT big final score:',f1_final_score)

(100, 768) (100, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT big final score: 0.43


In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)         # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.4525
              precision    recall  f1-score   support

  akadēmisks       1.00      0.28      0.44        25
        citi       0.32      1.00      0.49        25
 darba laiks       1.00      0.44      0.61        25
   dokumenti       1.00      0.16      0.28        25

    accuracy                           0.47       100
   macro avg       0.83      0.47      0.45       100
weighted avg       0.83      0.47      0.45       100

[[ 7 18  0  0]
 [ 0 25  0  0]
 [ 0 14 11  0]
 [ 0 21  0  4]]
[[0.28 0.72 0.   0.  ]
 [0.   1.   0.   0.  ]
 [0.   0.56 0.44 0.  ]
 [0.   0.84 0.   0.16]]


<h2>Papildināto jautājumu izvērtēšana</h2>

In [0]:
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets/BERT_encoded'
papildinatie = pd.read_csv('2020_05_10_encoded_BERT_big.csv', index_col=0)
print(papildinatie.shape)
papildinatie['labels'].value_counts()
col_count = len(papildinatie.columns) - 1
features = papildinatie.iloc[:,:col_count]
target = papildinatie.iloc[:,col_count:]
print(features.shape, target.shape)

/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets/BERT_encoded
(23391, 769)
(23391, 768) (23391, 1)


In [0]:
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
y_test = Y.to_numpy()

model_scores = []
counter = 0
for i in range(10):
  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(features, target,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT big final score:',f1_final_score)

(100, 768) (100, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT big final score: 0.71


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.7315
              precision    recall  f1-score   support

  akadēmisks       1.00      0.72      0.84        25
        citi       0.48      0.96      0.64        25
 darba laiks       1.00      0.48      0.65        25
   dokumenti       0.90      0.72      0.80        25

    accuracy                           0.72       100
   macro avg       0.84      0.72      0.73       100
weighted avg       0.84      0.72      0.73       100

[[18  6  0  1]
 [ 0 24  0  1]
 [ 0 13 12  0]
 [ 0  7  0 18]]
[[0.72 0.24 0.   0.04]
 [0.   0.96 0.   0.04]
 [0.   0.52 0.48 0.  ]
 [0.   0.28 0.   0.72]]


<h2>UnderSampling izvērtēšana</h2>

In [0]:
from imblearn.under_sampling import NearMiss
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

model_scores = []
counter = 0
for i in range(10):
  undersampler = NearMiss()
  X_res, y_res = undersampler.fit_resample(features, target)                   # Veicam Unersampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT big final score:',f1_final_score)

(100, 768) (100, 1)
(23391, 768) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT big final score: 0.68


In [0]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)         # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.67
              precision    recall  f1-score   support

  akadēmisks       0.75      0.96      0.84        25
        citi       1.00      0.08      0.15        25
 darba laiks       0.75      0.96      0.84        25
   dokumenti       0.74      1.00      0.85        25

    accuracy                           0.75       100
   macro avg       0.81      0.75      0.67       100
weighted avg       0.81      0.75      0.67       100

[[24  0  0  1]
 [ 7  2  8  8]
 [ 1  0 24  0]
 [ 0  0  0 25]]
[[0.96 0.   0.   0.04]
 [0.28 0.08 0.32 0.32]
 [0.04 0.   0.96 0.  ]
 [0.   0.   0.   1.  ]]


<h2>OverSampling izvērtēšana</h2>

<h3>OverSampling apjoms = 100%</h3>

In [0]:
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 2*122
thisdict = {
  1: 23025,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)

model_scores = []
counter = 0
for i in range(10):
  undersampler = NearMiss()
  X_res, y_res = oversampler.fit_resample(features, target)                       # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT big final score:',f1_final_score)

(100, 768) (100, 1)
(23391, 768) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT big final score: 0.77


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.8036
              precision    recall  f1-score   support

  akadēmisks       1.00      1.00      1.00        25
        citi       0.56      0.96      0.71        25
 darba laiks       1.00      0.56      0.72        25
   dokumenti       0.94      0.68      0.79        25

    accuracy                           0.80       100
   macro avg       0.88      0.80      0.80       100
weighted avg       0.88      0.80      0.80       100

[[25  0  0  0]
 [ 0 24  0  1]
 [ 0 11 14  0]
 [ 0  8  0 17]]
[[1.   0.   0.   0.  ]
 [0.   0.96 0.   0.04]
 [0.   0.44 0.56 0.  ]
 [0.   0.32 0.   0.68]]


<h3>OverSampling apjoms = 200%</h3>

In [0]:
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 3*122
thisdict = {
  1: 23025,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)

model_scores = []
counter = 0
for i in range(10):
  undersampler = NearMiss()
  X_res, y_res = oversampler.fit_resample(features, target)                       # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT big final score:',f1_final_score)

(100, 768) (100, 1)
(23391, 768) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT big final score: 0.75


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.8106
              precision    recall  f1-score   support

  akadēmisks       1.00      0.88      0.94        25
        citi       0.56      0.96      0.71        25
 darba laiks       1.00      0.68      0.81        25
   dokumenti       0.94      0.68      0.79        25

    accuracy                           0.80       100
   macro avg       0.88      0.80      0.81       100
weighted avg       0.88      0.80      0.81       100

[[22  3  0  0]
 [ 0 24  0  1]
 [ 0  8 17  0]
 [ 0  8  0 17]]
[[0.88 0.12 0.   0.  ]
 [0.   0.96 0.   0.04]
 [0.   0.32 0.68 0.  ]
 [0.   0.32 0.   0.68]]


<h3>OverSampling apjoms = 300%</h3>

In [0]:
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 4*122
thisdict = {
  1: 23025,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)

model_scores = []
counter = 0
for i in range(10):
  undersampler = NearMiss()
  X_res, y_res = oversampler.fit_resample(features, target)                       # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT big final score:',f1_final_score)

(100, 768) (100, 1)
(23391, 768) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT big final score: 0.77


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

(23391, 768) (23391, 1)
[[    0     1     2     3]
 [  488 23025   488   488]]
Keras Done
Naive Bayes F1 score: 0.818
              precision    recall  f1-score   support

  akadēmisks       1.00      0.96      0.98        25
        citi       0.59      0.96      0.73        25
 darba laiks       1.00      0.48      0.65        25
   dokumenti       0.96      0.88      0.92        25

    accuracy                           0.82       100
   macro avg       0.89      0.82      0.82       100
weighted avg       0.89      0.82      0.82       100

[[24  1  0  0]
 [ 0 24  0  1]
 [ 0 13 12  0]
 [ 0  3  0 22]]
[[0.96 0.04 0.   0.  ]
 [0.   0.96 0.   0.04]
 [0.   0.52 0.48 0.  ]
 [0.   0.12 0.   0.88]]


<h2>Kombinētā Under & Over Sampling izvērtēšana</h2>

<h3>OverSampling apjoms = 100%</h3>

In [0]:
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 2*122
thisdict = {
  1: size,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)
undersampler = NearMiss()

model_scores = []
counter = 0
for i in range(10):
  X_res, y_res = undersampler.fit_resample(features, target)                      # Veicam Unersampling
  X_res, y_res = oversampler.fit_resample(X_res, y_res)                           # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT big final score:',f1_final_score)

(100, 768) (100, 1)
(23391, 768) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT big final score: 0.67


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.6802
              precision    recall  f1-score   support

  akadēmisks       0.86      0.96      0.91        25
        citi       1.00      0.08      0.15        25
 darba laiks       0.69      1.00      0.82        25
   dokumenti       0.74      1.00      0.85        25

    accuracy                           0.76       100
   macro avg       0.82      0.76      0.68       100
weighted avg       0.82      0.76      0.68       100

[[24  0  0  1]
 [ 4  2 11  8]
 [ 0  0 25  0]
 [ 0  0  0 25]]
[[0.96 0.   0.   0.04]
 [0.16 0.08 0.44 0.32]
 [0.   0.   1.   0.  ]
 [0.   0.   0.   1.  ]]


<h3>OverSampling apjoms = 200%</h3>

In [0]:
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 3*122
thisdict = {
  1: size,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)
undersampler = NearMiss()

model_scores = []
counter = 0
for i in range(10):
  X_res, y_res = undersampler.fit_resample(features, target)                      # Veicam Unersampling
  X_res, y_res = oversampler.fit_resample(X_res, y_res)                           # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT big final score:',f1_final_score)

(100, 768) (100, 1)
(23391, 768) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT big final score: 0.69


In [0]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)         # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

[[  0   1   2   3]
 [122 122 122 122]]
[[  0   1   2   3]
 [366 366 366 366]]
Keras Done
Naive Bayes F1 score: 0.7391
              precision    recall  f1-score   support

  akadēmisks       0.92      0.96      0.94        25
        citi       1.00      0.20      0.33        25
 darba laiks       0.76      1.00      0.86        25
   dokumenti       0.69      1.00      0.82        25

    accuracy                           0.79       100
   macro avg       0.84      0.79      0.74       100
weighted avg       0.84      0.79      0.74       100

[[24  0  0  1]
 [ 2  5  8 10]
 [ 0  0 25  0]
 [ 0  0  0 25]]
[[0.96 0.   0.   0.04]
 [0.08 0.2  0.32 0.4 ]
 [0.   0.   1.   0.  ]
 [0.   0.   0.   1.  ]]


<h3>OverSampling apjoms = 300%</h3>

In [0]:
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import BorderlineSMOTE
from keras import layers, models
from sklearn.metrics import f1_score
print(X.shape, Y.shape)
print(features.shape, target.shape)
y_test = Y.to_numpy()

size = 4*122
thisdict = {
  1: size,
  3: size,
  2: size,
  0: size
}
oversampler = BorderlineSMOTE(sampling_strategy=thisdict)
undersampler = NearMiss()

model_scores = []
counter = 0
for i in range(10):
  X_res, y_res = undersampler.fit_resample(features, target)                      # Veicam Unersampling
  X_res, y_res = oversampler.fit_resample(X_res, y_res)                           # Veicam Oversampling

  ### Keras Sequential ###
  model = models.Sequential()
  model.add(layers.Dense(256, activation='relu', input_shape=(col_count,)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(4, activation='softmax'))
  model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
  history = model.fit(X_res, y_res,
                      batch_size=64,
                      epochs=5,
                      verbose=0)
  y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
  keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
  f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)              # Fiksējam f1 vidējo rezultātu
  counter = counter + 1
  model_scores.append(f1_avg)
  print('Completed iteration #',counter)
f1_final_score = round(np.mean(model_scores),2)
print('BERT big final score:',f1_final_score)

(100, 768) (100, 1)
(23391, 768) (23391, 1)
Completed iteration # 1
Completed iteration # 2
Completed iteration # 3
Completed iteration # 4
Completed iteration # 5
Completed iteration # 6
Completed iteration # 7
Completed iteration # 8
Completed iteration # 9
Completed iteration # 10
BERT big final score: 0.67


In [0]:
from sklearn.metrics import plot_confusion_matrix, f1_score, classification_report, confusion_matrix
y_pred = model.predict(X)                                                       # Prasam modelim paredzēt testa datiem kategorijas
keras_y_pred = transformResults(y_pred)                                         # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
y_test = Y.to_numpy()
class_names = ['akadēmisks', 'citi', 'darba laiks', 'dokumenti']
naive_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)               # Fiksējam Naive Bayes f1 vidējo rezultātu
print('Naive Bayes F1 score:', naive_f1_avg)
print(classification_report(y_test, keras_y_pred, target_names=class_names))
print(confusion_matrix(y_test, keras_y_pred, normalize=None))
print(confusion_matrix(y_test, keras_y_pred, normalize='true'))

Naive Bayes F1 score: 0.7213
              precision    recall  f1-score   support

  akadēmisks       0.83      0.96      0.89        25
        citi       1.00      0.20      0.33        25
 darba laiks       0.83      0.96      0.89        25
   dokumenti       0.65      0.96      0.77        25

    accuracy                           0.77       100
   macro avg       0.83      0.77      0.72       100
weighted avg       0.83      0.77      0.72       100

[[24  0  0  1]
 [ 3  5  5 12]
 [ 1  0 24  0]
 [ 1  0  0 24]]
[[0.96 0.   0.   0.04]
 [0.12 0.2  0.2  0.48]
 [0.04 0.   0.96 0.  ]
 [0.04 0.   0.   0.96]]
